In [1]:
# image convolution and cross-correlation

import torch
from torch import nn

# define cross-correlation function
# = convolution
def cro_cor(X, kernel):
    h, w = kernel.shape
    y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(y.shape[0]):
        for j in range(y.shape[1]):
            y[i, j] = (X[i:i+h, j:j+w] * kernel).sum()
    return y

# test function with simple example
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
k = torch.tensor([[0.0, 1.0], [2.0, 3.0]])

y = cro_cor(X, k)
print(y)

tensor([[19., 25.],
        [37., 43.]])


In [2]:
# custom layer
class Conv2D(nn.Module):
    def __init__(self, kernel_size) -> None:
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size), requires_grad=True)
        self.bias = nn.Parameter(torch.zeros(1), requires_grad=True)
    
    def forward(self, X):
        return cro_cor(X, self.weight)#  + self.bias
    
# test edge
# ---------
# manully cross-correlation computation
X = torch.ones((6, 8))
X[:, 2:6] = 0
print(X)

kernel_edge = torch.tensor([[1.0, -1.0]])  # this kernel can only detect vertical edge
y = cro_cor(X, kernel_edge)
print(y)                                   # successfully detect edge

print(cro_cor(X.t(), kernel_edge))         # unsucceed
print(cro_cor(X.t(), kernel_edge.t()))     # succeed

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])
tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])
tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])
tensor([[ 0.,  0.,  0.,  0.,  0.,  0.],
        [ 1.,  1.,  1.,  1.,  1.,  1.],
        [ 0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.],
        [-1., -1., -1., -1., -1., -

In [3]:
# most of the time we cannot self make a kernel because we wont know the pattern
# so we have to learn the kernel to detect the possible edge
conv2d = nn.Conv2d(1, 1, kernel_size=(1, 2), bias=False)  # in_channel, out_channel = 1, 1 we suppose
conv2d_cus = Conv2D((1, 2))
# features and labels should be the same shape
X_copy = X.reshape((1, 1, 6, 8))   # features
y_copy = y.reshape((1, 1, 6, 7))   # labels
lr = 25e-3

n_epochs = 10
for i in range(n_epochs):
    y_hat = conv2d_cus(X)
    l = (y_hat - y) ** 2
    conv2d_cus.zero_grad()
    l.sum().backward()
    conv2d_cus.weight.data[:] -= conv2d_cus.weight.grad * lr
    # conv2d_cus.bias.data[:] -= conv2d_cus.bias.grad * lr
    if (i + 1) % 2 == 0:
        print(f'epoch {i+1}, loss {l.sum():.3f}')

print(conv2d_cus.weight.data.reshape((1, 2)))  # we successfully learn the kernel(like the manully created one)

epoch 2, loss 6.111
epoch 4, loss 1.434
epoch 6, loss 0.342
epoch 8, loss 0.082
epoch 10, loss 0.020
tensor([[ 0.9718, -0.9715]])
